In [13]:
dataFile = "/content/drive/MyDrive/avalonWHistoryfixed.tsv"

In [ ]:
#https://discuss.huggingface.co/t/save-load-and-do-inference-with-fine-tuned-model/76291/2

In [2]:
!pip install datasets
!pip install evaluate
!pip install transformers 4.48.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement 4.48.0 (from versions: none)
ERROR: No matching distribution found for 4.48.0


In [3]:
import transformers
from transformers import RobertaForSequenceClassification, AutoModelForSequenceClassification, AutoTokenizer, Trainer
import evaluate
import pandas as pd
import numpy as np
import datasets
from datasets import Dataset

In [4]:
#roberta
robertaMafia = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/robertaMafiawHistory/checkpoint-131")

#robertaUnidecor = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/models2/robertaunidecor")

robertaTokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base", use_fast=True)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
#mbert
mbertMafia = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/mbertMafiawhistory/checkpoint-393")

#mbertUnidecor = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/new models/mbertunidecor")

mbertTokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")



tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [6]:
def preprocess_mbert(examples):
    label_mapping = {"T": 0, "D": 1}

    res  = mbertTokenizer(examples["text"], padding=True, truncation=True, max_length=512)
    res['labels'] = [label_mapping[label] for label in examples['labels']]

    return res

In [7]:
def preprocess_roberta(examples):
    label_mapping = {"T": 0, "D": 1}

    res  = robertaTokenizer(examples["text"], padding=True, truncation=True, max_length=512)
    res['labels'] = [label_mapping[label] for label in examples['labels']]

    return res

In [14]:
#def dataFix(row):
#  return "Game history: " + row['history'] + "Detect deception for the following message: " + row['withName']

df=pd.read_csv(dataFile, sep='\t')

df = df.drop(["mid","persuasion", "deception","player","quest","turn","file", "Unnamed: 0.1", "Unnamed: 0", "msg", "withName", "history"], axis=1)
#df['msgAndHistory'] = df.apply(dataFix, axis = 1)
df = df.rename(columns={'label': 'labels','msgAndHistory': 'text'})
print(df)

humanEval = Dataset.from_pandas(df)

     labels                                               text
0         T  Game history: "" Detect deception for the foll...
1         T  Game history: "player-4: random given no infor...
2         T  Game history: "player-4: random given no infor...
3         T  Game history: "player-4: random given no infor...
4         T  Game history: "player-4: random given no infor...
...     ...                                                ...
2379      T  Game history: "player-3: whelp, no sure who to...
2380      T  Game history: "player-3: whelp, no sure who to...
2381      T  Game history: "player-3: whelp, no sure who to...
2382      T  Game history: "player-3: whelp, no sure who to...
2383      T  Game history: "player-3: whelp, no sure who to...

[2384 rows x 2 columns]


In [ ]:
#https://discuss.huggingface.co/t/how-do-i-evaluate-a-pretrained-model-on-a-test-dataset/15099/2

In [20]:
acc = evaluate.load("accuracy")
confusion_metric = evaluate.load("confusion_matrix")
f1_metric = evaluate.load("f1")
recall = evaluate.load('recall')
precision = evaluate.load('precision')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    pred = np.argmax(predictions, axis=-1)

    df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/llm decetion/predictionsfixed.tsv", sep='\t')
    df['mberthistorypred'] = pred
    df['mberthistorylabelsT0D1'] = labels
    df.to_csv("/content/drive/MyDrive/Colab Notebooks/llm decetion/predictionsfixed.tsv", sep='\t')

    accuracy = acc.compute(predictions=pred, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=pred, references=labels)["f1"]
    cm = confusion_metric.compute(predictions=pred, references=labels)["confusion_matrix"]
    rc = recall.compute(predictions=pred, references=labels)["recall"]
    pr = precision.compute(predictions=pred, references=labels)["precision"]

    res = {
        "accuracy": accuracy,
        "f1": f1,
        "recall": rc,
        "recision": pr,
        "cm_true_positives": float(cm[0][0]),
        "cm_false_positives": float(cm[0][1]),
        "cm_false_negatives": float(cm[1][0]),
        "cm_true_negatives": float(cm[1][1]),
    }
    print(res)
    return res


def eval(model, data):
  trainer = Trainer(
      model=model,
      #args=training_args,
      #train_dataset=small_train_dataset,
      eval_dataset=data,
      compute_metrics=compute_metrics,
  )

  trainer.evaluate()



In [12]:
import os
os.environ['WANDB_MODE'] = 'disabled'

In [19]:
#roberta
encoded_evalR = humanEval.map(preprocess_roberta,  batched=True)
eval(robertaMafia, encoded_evalR)

Map:   0%|          | 0/2384 [00:00<?, ? examples/s]

{'accuracy': 0.8204697986577181, 'f1': 0.0, 'recall': 0.0, 'recision': 0.0, 'cm_true_positives': 1956.0, 'cm_false_positives': 0.0, 'cm_false_negatives': 428.0, 'cm_true_negatives': 0.0}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
#mbert
encoded_evalM = humanEval.map(preprocess_mbert, batched=True)
eval(mbertMafia, encoded_evalM)

Map:   0%|          | 0/2384 [00:00<?, ? examples/s]

A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


{'accuracy': 0.7273489932885906, 'f1': 0.12634408602150538, 'recall': 0.10981308411214953, 'recision': 0.14873417721518986, 'cm_true_positives': 1687.0, 'cm_false_positives': 269.0, 'cm_false_negatives': 381.0, 'cm_true_negatives': 47.0}
